Packages

In [1]:
import pandas as pd
from openpyxl import load_workbook

Parameters

In [2]:
Horizon = 2040
start_year = 2020
LeapYearsInvestment = 5
total_periods = int((Horizon - start_year)/LeapYearsInvestment)

bus_map = load_workbook('data_pypsa/bus_map30.xlsx').active
new_names_2 = [cell.value for cell in bus_map['A'][1:]]

General.xlsx

Generator.xlsx

In [ ]:
pypsa_tech = load_workbook('data_pypsa/technologies.xlsx').active

Node.xlsx

In [4]:
workbook = load_workbook('Data handler/bolivia_v1/Node.xlsx')
pypsa_load = load_workbook('data_pypsa/load_per_gen30.xlsx').active
load_dict = {row[0]: row[1] for row in pypsa_load.iter_rows(min_row=2, values_only=True)}

for sheet in workbook.sheetnames:
    current_sheet = workbook[sheet]

    nodes_positions = {'A3': ('B3', 1), 'E3': ('F3', 5)}
    
    for node_cell_ref, (period_cell_ref, col_index) in nodes_positions.items():
        if current_sheet[node_cell_ref].value == 'Nodes':
            # Check if the cell next to "Nodes" contains "Period"
            if current_sheet[period_cell_ref].value == 'Period':
                # Logic for handling periods
                start_row = current_sheet[node_cell_ref].row + 1
                
                # Repeat names for each period
                for period_index in range(total_periods):
                    period_start_row = start_row + (period_index * len(new_names_2))
                    
                    # Fill names for the current period
                    for i, name in enumerate(new_names_2, start=period_start_row):
                        current_sheet.cell(row=i, column=col_index, value=name)
                        current_sheet.cell(row=i, column=col_index + 1, value=period_index + 1)

                        if period_index == 0:
                            current_sheet.cell(row=i, column=col_index + 2, value=load_dict[name])
                            

                # Clear cells after the last period's names, if necessary
                last_name_row = start_row + (total_periods * len(new_names_2))
                for i in range(last_name_row, current_sheet.max_row + 1):
                    if current_sheet.cell(row=i, column=col_index).value or current_sheet.cell(row=i, column=col_index + 1).value:
                        current_sheet.cell(row=i, column=col_index).value = None
                        current_sheet.cell(row=i, column=col_index + 1).value = None
            else:
                max_row = current_sheet[node_cell_ref].row + len(new_names_2)
                # Fill names directly if no "Period" is next to "Nodes"
                for i, name in enumerate(new_names_2, start=current_sheet[node_cell_ref].row + 1):
                    current_sheet.cell(row=i, column=col_index, value=name)

                for i in range(max_row + 1, current_sheet.max_row + 1):
                    if current_sheet.cell(row=i, column=col_index).value is not None:
                        current_sheet.cell(row=i, column=col_index).value = None

workbook.save('Data handler/bolivia_v1/Node.xlsx')



Sets.xlsx

In [ ]:
file_path = 'Data handler/bolivia_v1/Sets.xlsx'
workbook = load_workbook(file_path)

# Nodes

node_sheet = workbook['Nodes']

max_row = node_sheet['A1'].row + len(new_names_2)
# Fill names directly if no "Period" is next to "Nodes"
for i, name in enumerate(new_names_2, start=node_sheet['A1'].row + 1):
    node_sheet.cell(row=i, column=1, value=name)

for i in range(max_row + 1, node_sheet.max_row + 1):
    if node_sheet.cell(row=i, column=1).value is not None:
        node_sheet.cell(row=i, column=1).value = None


# OffshoreNodes

offshore_node_sheet = workbook['OffshoreNodes']
max_row = offshore_node_sheet.max_row

for i in range(1, max_row):
    if offshore_node_sheet.cell(row=i + 1, column=1).value is not None:
        offshore_node_sheet.cell(row=i + 1, column=1).value = None


# Horizon


# Storage


# Technology


# Generators


# LineType


# HourOfSeason


# StorageOfNodes


# DirectionalLines
line_connections = workbook['DirectionalLines']
pypsa_connections = load_workbook('data_pypsa/lines30.xlsx').active

for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    value_a, value_b = row[1], row[2]

    line_connections.cell(row=index + 3, column=1, value=value_a)
    line_connections.cell(row=index + 3, column=2, value=value_b)

max_row = 2 + len(pypsa_connections['A'])
for i in range(max_row + 1, line_connections.max_row + 1):
    for col in range(1, 4): 
        if line_connections.cell(row=i, column=col).value is not None:
            line_connections.cell(row=i, column=col).value = None

# LineTypeOfDirectionalLines
line_type_connections = workbook['LineTypeOfDirectionalLines']

for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    value_a, value_b = row[1], row[2]

    line_type_connections.cell(row=index + 3, column=1, value=value_a)
    line_type_connections.cell(row=index + 3, column=2, value=value_b)
    line_type_connections.cell(row=index + 3, column=3, value='HVAC_OverheadLine')

for i in range(max_row + 1, line_type_connections.max_row + 1):
    for col in range(1, 4): 
        if line_type_connections.cell(row=i, column=col).value is not None:
            line_type_connections.cell(row=i, column=col).value = None




# GeneratorsOfNode


# GeneratorsOfTechnology


# Coords

coord_sheet = workbook['Coords']
pypsa_coords = load_workbook('data_pypsa/buses_EMPIRE.xlsx').active
max_row = 4 + len(new_names_2)

for index, row in enumerate(pypsa_coords.iter_rows(min_row=2, values_only=True), start=1):
    node, x, y = row[0], row[3], row[4]
    coord_sheet.cell(row=index + 3, column=1, value=node)
    coord_sheet.cell(row=index + 3, column=2, value=y)
    coord_sheet.cell(row=index + 3, column=3, value=x)

for i in range(max_row, coord_sheet.max_row + 1):
    for col in range(1, 4): 
        if coord_sheet.cell(row=i, column=col).value is not None:
            coord_sheet.cell(row=i, column=col).value = None

workbook.save(file_path)